## Datasets Used

## 1. HDMA River Network

**Description**  
The river network used in this study is derived from the **Hydrologic Derivatives for Modeling and Applications (HDMA)** database.

**Citation**  
Verdin, K. L. (2017).  
*Hydrologic Derivatives for Modeling and Applications (HDMA) database.*  
U.S. Geological Survey data release.  

**Dataset Access**    
https://doi.org/10.5066/F7S180ZP

---

### 2. HydroLAKES (Version 1)

**Description**  
A global vector database of lakes and reservoirs, providing detailed information on lake shorelines, surface area, volume, depth estimates, and hydrological connectivity. HydroLAKES is widely used in global hydrology and water resources studies.

**Citation**  
Messager, M. L., Lehner, B., Grill, G., Nedeva, I., & Schmitt, O. (2016).  
*Estimating the volume and age of water stored in global lakes using a geostatistical approach.*  
**Nature Communications**, 7, 13603.  
https://doi.org/10.1038/ncomms13603

**Dataset Access**  
- HydroLAKES product page:  
  https://www.hydrosheds.org/products/hydrolakes

## Assigning parameters and folders

In [1]:
# outputfolder for where the files will be sitting
OutFolder = '/Users/shg096/Desktop/LakeRiverOut/HDMA/'

# location of MERIT-Basin bug fixed files
riv_file="/Volumes/F:/hydrography/HDMA/processed/hdma_global_stream.shp"
cat_file="/Volumes/F:/hydrography/HDMA/processed/hdma_global_catch_v2_*000000.gpkg/hdma_global_catch_v2_*000000.shp"

# location of HydroLAKES
lake_file = '/Volumes/F:/hydrography/hydrolakes/HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10.shp'

# HDMA regions
regions = ['3'] # '1' Asia, '2' Africa, '3' North America 

In [2]:
# load the needed packages
import os
import shutil
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from   riverlakenetwork import Utility, BurnLakes
import warnings; warnings.filterwarnings("ignore")

In [3]:
#load hydrolakeDataset
lake = gpd.read_file(lake_file) # read the hydrolake dataset
# merge lake Michigan and Huron as they are hydraulically connected
lake = Utility.FixHydroLAKESv1(lake, merge_lakes={"Michigan+Huron": [6, 8]})

for region in regions:
    
    # read the pfaf merit folder
    riv, cat = Utility.hdma_read_file (region,
                                       riv_file_template=riv_file,
                                       cat_file_template=cat_file)
    
    # conver the unit area for cat from m2 to km2
    cat["area_org"]=cat["area_org"] / 1e6

    # create folder to save
    region_base = f"region{region}"
    # create the folder if not existed
    org_folder = os.path.join(OutFolder, f"{region_base}_org")
    if os.path.isdir(org_folder):
        try:
            shutil.rmtree(org_folder)
        except OSError as e:
            raise RuntimeError(f"Failed to remove {org_folder}: {e}")
    os.makedirs(org_folder, exist_ok=True)
    
    # save riv, and cat
    riv.to_file(os.path.join(org_folder, "riv.gpkg"))
    cat.to_file(os.path.join(org_folder, "cat.gpkg"))
    
    # create the config and pass it to the Burn lake
    config = {
        "riv": riv,
        "riv_dict": {
            "COMID": {"col":"seg_id"},
            "NextDownCOMID": {"col":"Tosegment"},
            "length": {"col":"Length"},
            "uparea": {"col":"flow_acc","unit":"km2"}
        },
        "cat": cat,
        "cat_dict": {
            "COMID": {"col":"hruid"},
            "unitarea": {"col":"area_org","unit":"km2"},
        },
        "lake": lake,
        "lake_dict": {
            "LakeCOMID": {"col":"Hylak_id"},
            "unitarea": {"col":"Lake_area","unit":"km2"}
        },
    }

    # burn lakes into river network
    bl = BurnLakes(config)

    # create folder to save
    region_base = f"region{region}"
    # create the folder if not existed
    corrected_folder = os.path.join(OutFolder, f"{region_base}_corrected")
    if os.path.isdir(corrected_folder):
        try:
            shutil.rmtree(corrected_folder)
        except OSError as e:
            raise RuntimeError(f"Failed to remove {corrected_folder}: {e}")
    os.makedirs(corrected_folder, exist_ok=True)

    # save riv, cat, and lake
    bl.riv.to_file(os.path.join(corrected_folder, "riv.gpkg"))
    bl.cat.to_file(os.path.join(corrected_folder, "cat.gpkg"))
    bl.lake.to_file(os.path.join(corrected_folder, "lake.gpkg"))

=== Input loader started at : 2025-12-23 19:51:41  ===
riv: Loaded
riv_dict: {'COMID': {'col': 'seg_id'}, 'NextDownCOMID': {'col': 'Tosegment'}, 'length': {'col': 'Length'}, 'uparea': {'col': 'flow_acc', 'unit': 'km2'}}
cat: Loaded
cat_dict: {'COMID': {'col': 'hruid'}, 'unitarea': {'col': 'area_org', 'unit': 'km2'}}
lake: Loaded
lake_dict: {'LakeCOMID': {'col': 'Hylak_id'}, 'unitarea': {'col': 'Lake_area', 'unit': 'km2'}}
=== Input loader finished at: 2025-12-23 19:51:44  ===
=== Input loader took      : 0:00:03.192804  ===========================
=== Input checker started at : 2025-12-23 19:51:44  ===
Subbasin and lake area units are consistent: km2
riv CRS: EPSG:4326
cat CRS: EPSG:4326
lake CRS: EPSG:4326
✅ No loop detected in network topology
=== Input checker finished at: 2025-12-23 19:51:46  ===
=== Input checker took      : 0:00:02.081476  ===========================
=== Resolving lakes started at : 2025-12-23 19:51:46  ===
==== Number of lakes after subsetting: 990183 ====
==== 